# Fermipy

Notes on the [fermipy quickstart guide](https://fermipy.readthedocs.io/en/latest/quickstart.html) and on the notebook tutorials.

**Contents**
* [Quickstart guide](#Quickstart-guide)
* [Creating an Analysis Script](#Creating-an-Analysis-Script)
* [Extracting Analysis Results](#Extracting-Analysis-Results)
* [*Extra*: queries and download data with astroquery](#Querying-and-Downloading-data-with-astroquery)

**References**
* [Mapping on the HEALPix grid - Calabretta *et al.*](https://fits.gsfc.nasa.gov/wcs/mhg_20050615.pdf)
* [The HEALPix Primer - Górski *et al.*](https://healpix.jpl.nasa.gov/pdf/intro.pdf)
* [Data Analysis on the Sphere - Sureau *et al.*](http://ada7.cosmostat.org/ADA7_Tutorial_6.pdf)
* [Galactic Interstellar Emission Model for the 4FGL Catalog Analysis - *Fermi*-LAT Collaboration](https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/aux/4fgl/Galactic_Diffuse_Emission_Model_for_the_4FGL_Catalog_Analysis.pdf)

## Quickstart guide

[**&uarr; Return to Contents**](#Fermipy)

In [1]:
% cd /home/mpotto/Pesquisa/astroph/tutorials/fermipy/mkn421

/home/mpotto/Pesquisa/astroph/tutorials/fermipy/mkn421


### Create a Configuration file

Check the syntax on the page [Configuration](https://fermipy.readthedocs.io/en/latest/config.html#config), [Cicerone Glossary](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Glossary.html) and other Cicerone threads for more info. Configuration files defines the data selection and analysis parameters. 

Our documented configuration file is ```config.yaml```

```yaml
data:
# The data section defines the input data files for the analysis.

  evfile : ft1.lst      # Path to FT1 files or list or list of files.          
  scfile : ft2.fits     # Path to FT2 (spacecraft) file. The spacecraft FITS file with the time history of the spacecraft position, LAT orientation and LAT livetime. Also called an FT2 file.
  ltcube : ltcube.fits  # Path to livetime cube.

binning:
# The binning section controls the spatial and binning of the data.

  roiwidth   : 10.0     # width of the ROI in degrees.
  binsz      : 0.1      # spatial bin size in degrees.
  binsperdec : 8        # number of bins per decade.

selection :
# The selection section collects parameters relatedd to the data selection and target definition. The majority of the parameters are arguments to gtselect and gtmktime.

  emin : 100            # Minimum energy (MeV)
  emax : 316227.76      # Maximum energy (MeV)
  zmax    : 90          # Maximum zenith angle.
  evclass : 128         # Event class selection.
  evtype  : 3           # Event type selection according to the IRF (in our case P8R2, Pass 8 Release 2)
  tmin    : 239557414   # Minimum time (MET, Mission Elapsed Time)
  tmax    : 428903014   # Maximum time (MET)
  filter  : null        # Filter string for gtmktime selection.
  target  : 'mkn421'     # Object on which to center the ROI. Has precedence over ra/dec or glon/glat

gtlike:
# The gtlike section controls the setup of the likelihood analysis 

  edisp : True                              # Enable correction for energy dispersion.
  irfs : 'P8R2_SOURCE_V6'                   # Set the IRF (Instrument Response Function) string.
  edisp_disable : ['isodiff','galdiff']     # List of sources for which edisp correction should be disabled.

model:
# The model section collects options that control the inclusion of point-source and difuse
# components in the model. 'galdiff' and 'isodiff' sets the templates for the Galactic IEM
# (Interstellar emission model) and istropic diffuse.

  src_roiwidth : 15.0                                   # Width of square region in degrees centered on the ROI that selects sources for inclusion in the model. If this parameter is none  then no selection is applied. This selection will be ORed with the src_radius selection.
  galdiff  : '$FERMI_DIFFUSE_DIR/gll_iem_v07.fits'      # Set the path to one or more galactic IEM mapcubes. A separate component will be generated for each item in this list.
  isodiff  : 'iso_P8R2_SOURCE_V6_v06.txt'               # Set the path to one or more isotropic templates. A separate component will be generated for each item in this list.
  catalogs : ['4FGL']                                   # List of catalogs that will be merged to form a master analysis catalog from which sources will be drawn. FGL stands for Fermi Gamma-ray LAT. 1FGL was released at 1 year, 2FGL at 2 year and 3FGL at 4 year. 4FGL was released on February 2019 and is based on 8 years (50 MeV - 1 TeV range).

#components:
# The components section gives the user the option to combine multiple data selections into a joint likelihood. It contains a list of dictionaries with the same hierarchy as the root analysis configuration. Each element of the list defines the analysis parameters for an independent sub-selection of the data. Any component not defined in the dictionary default to the value defined in the root config. 

# Example (changing event types in the IRF)

#    - { selection: { evtype : 4} }      # Set the evtype to PSF0 
#    - { selection: { evtype : 8} }      # PSF1
#    - { selection: { evtype : 16} }     # PSF2
#    - { selection: { evtype : 32} }     # PSF3
```

## Creating an Analysis Script

[**&uarr; Return to Contents**](#Fermipy)

In [20]:
from fermipy.gtanalysis import GTAnalysis

In [21]:
gta = GTAnalysis('config.yaml', logging={'verbosity': 3})

2020-04-08 14:55:49 INFO    GTAnalysis.__init__(): 
--------------------------------------------------------------------------------
fermipy version 0.18.1 
ScienceTools version unknown


Data preparation and response calculations (selecting data, creating counts and exposure maps, *etc.*). This step is *cached* so subsequent calls will run much faster.

In [22]:
gta.setup()

2020-04-08 14:55:50 INFO    GTAnalysis.setup(): Running setup.
2020-04-08 14:55:50 INFO    GTBinnedAnalysis.setup(): Running setup for component 00
2020-04-08 14:55:50 INFO    GTBinnedAnalysis._select_data(): Skipping data selection.
2020-04-08 14:55:50 INFO    GTBinnedAnalysis.setup(): Using external LT cube.
2020-04-08 14:55:51 INFO    GTBinnedAnalysis._create_expcube(): Skipping gtexpcube.
2020-04-08 14:55:51 INFO    GTBinnedAnalysis._create_srcmaps(): Skipping gtsrcmaps.
2020-04-08 14:55:51 INFO    GTBinnedAnalysis.setup(): Finished setup for component 00
2020-04-08 14:55:51 INFO    GTBinnedAnalysis._create_binned_analysis(): Creating BinnedAnalysis for component 00.
2020-04-08 14:56:03 INFO    GTAnalysis.setup(): Initializing source properties
2020-04-08 14:56:03 INFO    GTAnalysis.setup(): Finished setup.


Loop over all model components in the ROI and fit their normalization and spectral shape parameters. Also computer the TS (test statistic) of all sources which can be useful to identify weak sources. 

In [9]:
gta.optimize();

2020-04-06 13:01:55 INFO    GTAnalysis.optimize(): Starting


Joint fit  ['isodiff', 'galdiff', '4FGL J1104.4+3812', '4FGL J1112.5+3448', '4FGL J1127.8+3618']


/home/mpotto/anaconda3/envs/fermi/lib/python2.7/site-packages/scipy/interpolate/fitpack2.py:224: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


Fitting shape 4FGL J1104.4+3812 TS:  86171.358
Fitting shape isodiff TS:  14504.756
Fitting shape galdiff TS:   3572.598
Fitting shape 4FGL J1120.8+4212 TS:   1100.555
Fitting shape 4FGL J1112.5+3448 TS:    947.669
Fitting shape 4FGL J1127.8+3618 TS:    808.873
Fitting shape 4FGL J1100.3+4020 TS:    163.682
Fitting shape 4FGL J1128.8+3757 TS:    153.984
Fitting shape 4FGL J1129.1+3703 TS:    114.334
Fitting shape 4FGL J1101.4+4108 TS:     90.001
Fitting shape 4FGL J1131.0+3815 TS:     77.382
Fitting shape 4FGL J1051.4+3942 TS:     74.626
Fitting shape 4FGL J1101.5+3904 TS:     60.936
Fitting shape 4FGL J1033.1+4115 TS:     44.325
Fitting shape 4FGL J1109.6+3735 TS:     42.905
Fitting shape 4FGL J1105.8+3944 TS:     27.748


2020-04-06 13:02:10 INFO    GTAnalysis.optimize(): Finished
2020-04-06 13:02:10 INFO    GTAnalysis.optimize(): LogLike: -77165.495265 Delta-LogLike: 77.276440
2020-04-06 13:02:10 INFO    GTAnalysis.optimize(): Execution time: 14.38 s


Print optimization results (notice the fitted spectrum type and ts).

In [10]:
gta.print_roi()

2020-04-06 13:02:28 INFO    GTAnalysis.print_roi(): 
name                SpatialModel   SpectrumType     offset        ts       npred
--------------------------------------------------------------------------------
4FGL J1104.4+3812   PointSource    LogParabola       0.000  94758.00     29566.0
4FGL J1101.5+3904   PointSource    PowerLaw          1.038     61.32      1344.3
4FGL J1109.6+3735   PointSource    PowerLaw          1.192     43.90       121.8
4FGL J1105.8+3944   PointSource    PowerLaw          1.558     28.81       244.7
4FGL J1106.7+3623   PointSource    PowerLaw          1.863      5.44       336.8
4FGL J1100.3+4020   PointSource    PowerLaw          2.272    163.80       319.9
4FGL J1054.2+3926   PointSource    PowerLaw          2.343     24.93       299.2
4FGL J1111.0+3542   PointSource    PowerLaw          2.820     12.54       109.7
4FGL J1051.4+3942   PointSource    PowerLaw          2.942     77.00       155.9
4FGL J1101.4+4108   PointSource    PowerLaw          2.9

By default all models parameters are initially fixed. We can then free or fix parameters using the ```free_source()``` or ```free_sources()``` methods. In the following, we free the normalization of catalog sources within 3º of the ROI and free the galactic and isotropic diffuse components.

In [7]:
gta.free_sources(distance=3.0, pars='norm');

2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1104.4+3812     : ['norm']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1101.5+3904     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1109.6+3735     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1105.8+3944     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1106.7+3623     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1100.3+4020     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1054.2+3926     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1111.0+3542     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J10

In [8]:
gta.free_source('galdiff')
gta.free_source('isodiff')

2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for galdiff               : ['Index']


We can also free sources on the basis of their current TS and Npred (this is the number of photons used in the fit).

In [9]:
gta.free_sources(minmax_ts=[10, None], pars='norm');
gta.free_sources(minmax_ts=[None, 10], free=False, pars='norm');
gta.free_sources(minmax_npred=[10, 100], free=False, pars='norm');

2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1112.5+3448     : ['norm']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1041.7+3902     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1128.8+3757     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1127.8+3618     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1129.1+3703     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1120.8+4212     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1131.0+3815     : ['norm']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1033.1+4115     : ['Prefactor']
2020-03-16 13:44:43 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1139.0+

In [10]:
gta.fit();

2020-03-16 13:44:43 INFO    GTAnalysis.fit(): Starting fit.
2020-03-16 13:45:03 INFO    GTAnalysis.fit(): Fit returned successfully. Quality:   3 Status:   0
2020-03-16 13:45:03 INFO    GTAnalysis.fit(): LogLike:   -77162.266 DeltaLogLike:        3.229 


In [11]:
gta.write_roi('fit_model/fit_model')

2020-03-16 13:45:03 INFO    GTBinnedAnalysis.write_xml(): Writing /home/mpotto/Pesquisa/astroph/tutorials/fermipy/mkn421/fit_model_00.xml...
2020-03-16 13:45:03 INFO    GTAnalysis.write_fits(): Writing /home/mpotto/Pesquisa/astroph/tutorials/fermipy/mkn421/fit_model.fits...
2020-03-16 13:45:05 INFO    GTAnalysis.write_roi(): Writing /home/mpotto/Pesquisa/astroph/tutorials/fermipy/mkn421/fit_model.npy...


Once we have optimized our model for the ROI we can use ```residmap()``` and ```tsmap()``` to asses the fit quality and look for new sources.

In [12]:
# Dictionary defining the spatial/spectral parameters of the test source
model = {'SpatialModel' : 'PointSource', 'Index' : 2.0,
         'SpectrumType' : 'PowerLaw'}

m0 = gta.residmap('fit_model', model=model, make_plots=True)
m1 = gta.tsmap('fit_model', model=model, make_plots=True)

2020-03-16 13:45:05 INFO    GTAnalysis.residmap(): Generating residual maps
2020-03-16 13:45:05 INFO    GTAnalysis.add_source(): Adding source residmap_testsource
2020-03-16 13:45:07 INFO    GTAnalysis.delete_source(): Deleting source residmap_testsource
2020-03-16 13:45:09 INFO    GTAnalysis.residmap(): Finished residual maps
2020-03-16 13:45:09 INFO    GTAnalysis.residmap(): Execution time: 4.10 s
2020-03-16 13:45:09 INFO    GTAnalysis.tsmap(): Generating TS map
2020-03-16 13:45:11 INFO    GTAnalysis._make_tsmap_fast(): Fitting test source.
2020-03-16 13:45:29 INFO    GTAnalysis.tsmap(): Finished TS map
2020-03-16 13:45:29 INFO    GTAnalysis.tsmap(): Execution time: 19.22 s


By default, calls to ```fit()``` will execute a global spectral fit over the entire energy range of the analysis. To extract a bin-by-bin flux spectrum (SED or spectral energy distribution) you can call ```sed()``` method with the source name.

In [13]:
gta.sed('mkn421', make_plots=True);

2020-03-16 13:45:29 INFO    GTAnalysis.sed(): Computing SED for 4FGL J1104.4+3812
2020-03-16 13:45:39 INFO    GTAnalysis._make_sed(): Fitting SED
2020-03-16 13:45:39 INFO    GTAnalysis.free_source(): Fixing parameters for galdiff               : ['Index']
2020-03-16 13:45:45 INFO    GTAnalysis.sed(): Finished SED
2020-03-16 13:45:45 INFO    GTAnalysis.sed(): Execution time: 16.74 s


## Extracting Analysis Results

[**&uarr; Return to Contents**](#Fermipy)

In [14]:
import numpy as np

Loading from ```.npy``` format file.

In [15]:
c = np.load('fit_model.npy').flat[0]

In [16]:
c.keys()

['roi', 'stversion', 'version', 'config', 'sources']

In [17]:
c['sources'].keys()[:10]

['4FGL J1101.5+3904',
 '4FGL J1129.5+3034',
 '4FGL J1128.8+3757',
 '4FGL J1105.8+3944',
 '4FGL J1138.2+4115',
 '4FGL J1100.3+4020',
 '4FGL J1039.2+3258',
 '4FGL J1051.6+3253',
 '4FGL J1139.0+4033',
 '4FGL J1106.7+3623']

Information about individual sources in the ROI can be gathered in a catalog FITS file.

In [18]:
from astropy.table import Table

In [19]:
tab = Table.read('fit_model.fits')

In [20]:
tab[['name', 'class', 'ts', 'npred', 'flux']].to_pandas().head(10)

,name,class,ts,npred,flux
0,4FGL J1104.4+3812,BLL,86584.887885,29653.653811,1.999230e-07
1,4FGL J1101.5+3904,bcu,44.926957,1320.780916,1.315601e-08
2,4FGL J1109.6+3735,bll,44.199183,120.808769,7.998021e-10
3,4FGL J1105.8+3944,bll,28.509169,242.528664,1.983474e-09
4,4FGL J1106.7+3623,,5.441834,336.755649,3.513795e-09
5,4FGL J1100.3+4020,bll,166.705277,323.441088,2.302102e-09
6,4FGL J1054.2+3926,bcu,24.775421,301.199487,2.671951e-09
7,4FGL J1111.0+3542,bll,12.782255,108.831085,9.124106e-10
8,4FGL J1051.4+3942,bll,78.725214,157.072765,1.074298e-09
9,4FGL J1101.4+4108,bll,95.072888,157.698643,1.076507e-09


In [21]:
tab[['param_names','param_values','param_errors']][0]

param_names [10],param_values [10],param_errors [10]
str32,float64,float64
norm ..,1.89400602234e-11 .. nan,1.58045230647e-13 .. nan


## Reloading from a Previous State

[**&uarr; Return to Contents**](#Fermipy)

Construct a new instance of ```GTAnalysis``` from a previously saved results file.

In [22]:
gta_reloaded = GTAnalysis.create('fit_model.npy')

2020-03-16 13:45:46 INFO    GTAnalysis.__init__(): 
--------------------------------------------------------------------------------
fermipy version 0.18.1 
ScienceTools version unknown
2020-03-16 13:45:47 INFO    GTAnalysis.setup(): Running setup.
2020-03-16 13:45:47 INFO    GTBinnedAnalysis.setup(): Running setup for component 00
2020-03-16 13:45:47 INFO    GTBinnedAnalysis._select_data(): Skipping data selection.
2020-03-16 13:45:47 INFO    GTBinnedAnalysis.setup(): Using external LT cube.
2020-03-16 13:45:48 INFO    GTBinnedAnalysis._create_expcube(): Skipping gtexpcube.
2020-03-16 13:45:48 INFO    GTBinnedAnalysis._create_srcmaps(): Skipping gtsrcmaps.
2020-03-16 13:45:48 INFO    GTBinnedAnalysis.setup(): Finished setup for component 00
2020-03-16 13:45:48 INFO    GTBinnedAnalysis._create_binned_analysis(): Creating BinnedAnalysis for component 00.
2020-03-16 13:46:01 INFO    GTAnalysis.setup(): Finished setup.
2020-03-16 13:46:01 INFO    GTAnalysis.load_roi(): Loading ROI file: /

Or load the same instance of ```GTAnalysis``` on a previous state saved via ```write_roi()```.

In [23]:
gta = GTAnalysis('config.yaml')

2020-03-16 13:46:15 INFO    GTAnalysis.__init__(): 
--------------------------------------------------------------------------------
fermipy version 0.18.1 
ScienceTools version unknown


In [24]:
gta.setup()

2020-03-16 13:46:16 INFO    GTAnalysis.setup(): Running setup.
2020-03-16 13:46:16 INFO    GTBinnedAnalysis.setup(): Running setup for component 00
2020-03-16 13:46:16 INFO    GTBinnedAnalysis._select_data(): Skipping data selection.
2020-03-16 13:46:16 INFO    GTBinnedAnalysis.setup(): Using external LT cube.
2020-03-16 13:46:17 INFO    GTBinnedAnalysis._create_expcube(): Skipping gtexpcube.
2020-03-16 13:46:17 INFO    GTBinnedAnalysis._create_srcmaps(): Skipping gtsrcmaps.
2020-03-16 13:46:17 INFO    GTBinnedAnalysis.setup(): Finished setup for component 00
2020-03-16 13:46:17 INFO    GTBinnedAnalysis._create_binned_analysis(): Creating BinnedAnalysis for component 00.
2020-03-16 13:46:27 INFO    GTAnalysis.setup(): Initializing source properties
2020-03-16 13:46:27 INFO    GTAnalysis.setup(): Finished setup.


*Save a pre-fit state*

In [25]:
gta.write_roi('prefit_model')

2020-03-16 13:46:27 INFO    GTBinnedAnalysis.write_xml(): Writing /home/mpotto/Pesquisa/astroph/tutorials/fermipy/mkn421/prefit_model_00.xml...
2020-03-16 13:46:27 INFO    GTAnalysis.write_fits(): Writing /home/mpotto/Pesquisa/astroph/tutorials/fermipy/mkn421/prefit_model.fits...
2020-03-16 13:46:29 INFO    GTAnalysis.write_roi(): Writing /home/mpotto/Pesquisa/astroph/tutorials/fermipy/mkn421/prefit_model.npy...


In [26]:
gta.print_roi()

2020-03-16 13:46:29 INFO    GTAnalysis.print_roi(): 
name                SpatialModel   SpectrumType     offset        ts       npred
--------------------------------------------------------------------------------
4FGL J1104.4+3812   PointSource    LogParabola       0.000       nan     29970.1
4FGL J1101.5+3904   PointSource    PowerLaw          1.038       nan      1451.6
4FGL J1109.6+3735   PointSource    PowerLaw          1.192       nan       148.3
4FGL J1105.8+3944   PointSource    PowerLaw          1.558       nan       210.1
4FGL J1106.7+3623   PointSource    PowerLaw          1.863       nan       584.4
4FGL J1100.3+4020   PointSource    PowerLaw          2.272       nan       338.2
4FGL J1054.2+3926   PointSource    PowerLaw          2.343       nan       251.9
4FGL J1111.0+3542   PointSource    PowerLaw          2.820       nan       157.7
4FGL J1051.4+3942   PointSource    PowerLaw          2.942       nan       146.6
4FGL J1101.4+4108   PointSource    PowerLaw          2.9

*Do some modifications*

In [27]:
gta.free_source('mkn421')

2020-03-16 13:46:29 INFO    GTAnalysis.free_source(): Freeing parameters for 4FGL J1104.4+3812     : ['norm', 'alpha', 'beta']


In [28]:
gta.fit();

2020-03-16 13:46:29 INFO    GTAnalysis.fit(): Starting fit.
2020-03-16 13:46:29 INFO    GTAnalysis.fit(): Fit returned successfully. Quality:   3 Status:   0
2020-03-16 13:46:29 INFO    GTAnalysis.fit(): LogLike:   -77229.850 DeltaLogLike:       12.921 


In [29]:
gta.print_roi()

2020-03-16 13:46:29 INFO    GTAnalysis.print_roi(): 
name                SpatialModel   SpectrumType     offset        ts       npred
--------------------------------------------------------------------------------
4FGL J1104.4+3812   PointSource    LogParabola       0.000  95876.87     30604.8
4FGL J1101.5+3904   PointSource    PowerLaw          1.038       nan      1451.6
4FGL J1109.6+3735   PointSource    PowerLaw          1.192       nan       148.3
4FGL J1105.8+3944   PointSource    PowerLaw          1.558       nan       210.1
4FGL J1106.7+3623   PointSource    PowerLaw          1.863       nan       584.4
4FGL J1100.3+4020   PointSource    PowerLaw          2.272       nan       338.2
4FGL J1054.2+3926   PointSource    PowerLaw          2.343       nan       251.9
4FGL J1111.0+3542   PointSource    PowerLaw          2.820       nan       157.7
4FGL J1051.4+3942   PointSource    PowerLaw          2.942       nan       146.6
4FGL J1101.4+4108   PointSource    PowerLaw          2.9

*Reload pre-fit model*

In [30]:
gta.load_roi('prefit_model')

2020-03-16 13:46:29 INFO    GTAnalysis.load_roi(): Loading ROI file: /home/mpotto/Pesquisa/astroph/tutorials/fermipy/mkn421/prefit_model.npy
2020-03-16 13:46:29 INFO    GTBinnedAnalysis._create_binned_analysis(): Creating BinnedAnalysis for component 00.
2020-03-16 13:46:40 INFO    GTAnalysis.load_roi(): Finished Loading ROI


In [31]:
gta.print_roi()

2020-03-16 13:46:40 INFO    GTAnalysis.print_roi(): 
name                SpatialModel   SpectrumType     offset        ts       npred
--------------------------------------------------------------------------------
4FGL J1104.4+3812   PointSource    LogParabola       0.000       nan     29970.1
4FGL J1101.5+3904   PointSource    PowerLaw          1.038       nan      1451.6
4FGL J1109.6+3735   PointSource    PowerLaw          1.192       nan       148.3
4FGL J1105.8+3944   PointSource    PowerLaw          1.558       nan       210.1
4FGL J1106.7+3623   PointSource    PowerLaw          1.863       nan       584.4
4FGL J1100.3+4020   PointSource    PowerLaw          2.272       nan       338.2
4FGL J1054.2+3926   PointSource    PowerLaw          2.343       nan       251.9
4FGL J1111.0+3542   PointSource    PowerLaw          2.820       nan       157.7
4FGL J1051.4+3942   PointSource    PowerLaw          2.942       nan       146.6
4FGL J1101.4+4108   PointSource    PowerLaw          2.9

## Querying and Downloading data with astroquery

[**&uarr; Return to Contents**](#Fermipy)

Query using astroquery Fermi API.

In [32]:
from astroquery import fermi

/home/mpotto/anaconda3/envs/fermi/lib/python2.7/site-packages/astroquery/fermi/__init__.py:37: UserWarning: Experimental: Fermi-LAT has not yet been refactored to have its API match the rest of astroquery.
  warnings.warn("Experimental: Fermi-LAT has not yet been refactored to have "


In [33]:
result = fermi.FermiLAT.query_object(name_or_coords='238.929,11.1901', 
                                     searchradius='30',
                                     timesys='MET',
                                     obsdates='239557417, 256970880',
                                     energyrange_MeV='100,300000')

Download using ```urllib``` request.

In [34]:
from six.moves.urllib import request

In [35]:
image_ph1 = request.urlopen(result[-1])

Save the image as ```.fits```

In [ ]:
with open('test.fits', 'w') as image:
    image.write(image_ph1.read())

Verify the download by checking the PrimaryHDU header.

In [ ]:
from astropy.io import fits

In [ ]:
with fits.open('test.fits') as hdul:
    header = hdul[0].header